In [1]:
import tensorflow as tf
import numpy as np

In [2]:
import tensorflow_probability as tfp

In [3]:
# Load MNIST
(x_dev, y_dev), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

print('There are', len(x_dev), 'training images.')
print('There are', len(x_test), 'test images.')

There are 60000 training images.
There are 10000 test images.


In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((x_dev, y_dev)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [12]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import Model


class StochasticMLP(Model):
    
    def __init__(self, hidden_layer_sizes=[100], n_outputs=10):
        super(StochasticMLP, self).__init__()
        self.hidden_layer_sizes = hidden_layer_sizes
        self.fc_layers = [Dense(layer_size) for layer_size in hidden_layer_sizes]
        self.output_layer = Dense(n_outputs)

    
    def call(self, x):
        
        x = Flatten()(x)
        
        network = []
        
        for i, layer in enumerate(self.fc_layers):
            
            logits = layer(x)
            x = tfp.distributions.Bernoulli(logits=logits).sample()
            network.append(x)     
            
        final_logits = self.output_layer(x)
        
        return network
    
    def target_log_prob(self, x, h, y):
        
        x = Flatten()(x)
        
        h_current = h
        h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h_current]
        print(h_current)
        #print(np.asarray(h_current).shape)
        h_previous = [x] + h_current[:-1]
        
        nlog_prob = 0.
        
        for i, (cv, pv, layer) in enumerate(
            zip(h_current, h_previous, self.fc_layers)):
            
            ce = tf.nn.sigmoid_cross_entropy_with_logits( # only works for discretize version
                labels=cv, logits=layer(pv))
            nlog_prob += tf.reduce_sum(ce, axis = -1)
            
        nlog_prob += tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=tf.cast(y, tf.int32), logits=self.output_layer(h_current[-1]))
            
        return -1 * nlog_prob
    
    def target_log_prob2(self, x, h, y):
        
        x = Flatten()(x)
        #print(h)
        h_current = tf.split(h, self.hidden_layer_sizes, axis = 1)
        #print(h_current)
        h_previous = [x] + h_current[:-1]
        
        nlog_prob = 0.
        
        for i, (cv, pv, layer) in enumerate(
            zip(h_current, h_previous, self.fc_layers)):
            
            ce = tf.nn.sigmoid_cross_entropy_with_logits( # only works for discretize version
                labels=cv, logits=layer(pv))
            nlog_prob += tf.reduce_sum(ce, axis = -1)
            
        nlog_prob += tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=tf.cast(y, tf.int32), logits=self.output_layer(h_current[-1]))

        print(nlog_prob)
        return -1 * nlog_prob
    
    def run_chain(self, x, h, y):
        
        #print(h)
        
        h_current = h
        h_current = [tf.cast(h_i, dtype=tf.float32) for h_i in h_current]
        h_current = tf.concat([h_current[0], h_current[1]], axis=1)
        #print(h_current)
        
        #def tlp(args):
        #    return self.target_log_prob2(x, args, y)
        
        adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(tfp.mcmc.HamiltonianMonteCarlo(
            target_log_prob_fn = lambda v: self.target_log_prob2(x, v, y),
            num_leapfrog_steps = 2,
            step_size = pow(1000, -1/4)),
            num_adaptation_steps=int(100 * 0.8))
        
        #print(tf.size(tlp(h_current)))
        
        # Run the chain (with burn-in).
        samples = tfp.mcmc.sample_chain(
          num_results=1, # need one step
          num_burnin_steps=100, # set to 1000
          current_state=h_current,
          kernel=adaptive_hmc,
          trace_fn=None)
        
        #print(samples[0])
        new_state = tf.math.sign(tf.math.sign(samples[0]) - 1) + 1
        h_new = tf.split(new_state, self.hidden_layer_sizes, axis = 1)
        #print(h_new)
        
        return h_new

In [13]:
model = StochasticMLP(hidden_layer_sizes = [100, 50], n_outputs=10)

In [7]:
network = [model.call(images) for images, labels in train_ds]

In [14]:
hmc = [model.run_chain(images, net, labels) for (images, labels), net in zip(train_ds, network)]

tf.Tensor(
[3139.8477 4048.7666 3749.6665 5590.043  4632.822  4924.8047 4476.5015
 4603.577  3537.6099 4072.5303 4767.435  5437.341  4763.8657 2366.0874
 5010.219  4912.016  3055.324  3647.8638 5080.977  4919.0713 2582.5908
 4221.676  4854.228  3849.9749 3967.531  5618.1704 3283.8342 3578.8572
 3633.5974 6110.093  2953.6045 3405.1284], shape=(32,), dtype=float32)
tf.Tensor(
[3139.8477 4048.7666 3749.6665 5590.043  4632.822  4924.8047 4476.5015
 4603.577  3537.6099 4072.5303 4767.435  5437.341  4763.8657 2366.0874
 5010.219  4912.016  3055.324  3647.8638 5080.977  4919.0713 2582.5908
 4221.676  4854.228  3849.9749 3967.531  5618.1704 3283.8342 3578.8572
 3633.5974 6110.093  2953.6045 3405.1284], shape=(32,), dtype=float32)
tf.Tensor(
[ -9509.298  -16191.705  -11273.2705 -24206.309  -10407.161  -22379.13
  -9342.936  -20912.7    -12840.11   -18211.863  -15977.278  -22488.895
 -15261.624   -4128.4473 -28708.498  -13236.141   -5998.7246 -16241.167
 -12036.891  -22674.594   -6132.739  -1091

tf.Tensor(
[ -8143.861 -13137.626 -69825.25  -19943.447 -61484.84  -18373.525
  -7752.357 -17645.953 -10454.202 -15243.797 -13186.51  -19114.676
 -12671.442 -28068.3   -24449.213 -10509.127 -41077.656 -13721.579
 -73206.805 -18683.828 -36669.08   -9261.537 -15896.793 -59162.4
 -53267.1   -21453.258  -8534.395 -70815.58  -42506.42  -26560.695
 -36219.684 -11416.796], shape=(32,), dtype=float32)
tf.Tensor(
[ -8143.861 -13137.626 -69825.25  -19943.447 -61484.84  -18373.525
  -7752.357 -17645.953 -10454.202 -15243.797 -13186.51  -19114.676
 -12671.442 -28068.3   -24449.213 -10509.127 -41077.656 -13721.579
 -73206.805 -18683.828 -36669.08   -9261.537 -15896.793 -59162.4
 -53267.1   -21453.258  -8534.395 -70815.58  -42506.42  -26560.695
 -36219.684 -11416.796], shape=(32,), dtype=float32)
tf.Tensor(
[ -41426.258  -65213.516 -109636.766  -96572.15  -100657.836  -88954.65
  -44385.01   -84287.24   -52448.07   -73310.34   -66997.875  -92788.375
  -64678.773  -44830.766 -112340.66   -57007.934  

tf.Tensor(
[  -6597.2124  -75417.49    -68074.57    -17208.604   -59573.387
 -109641.305   -53383.76    -14705.871   -62428.152   -80414.305
  -74725.57    -16059.192   -72909.48    -27490.424   -20575.35
  -67774.29    -39913.918   -79250.57    -71256.95    -15949.062
  -35654.242   -58776.08    -94103.72    -57683.992   -51839.375
  -18169.535    -6839.1035  -68773.07    -41112.28    -22509.2
  -35473.77    -66644.05  ], shape=(32,), dtype=float32)
tf.Tensor(
[  -6597.2124  -75417.49    -68074.57    -17208.604   -59573.387
 -109641.305   -53383.76    -14705.871   -62428.152   -80414.305
  -74725.57    -16059.192   -72909.48    -27490.424   -20575.35
  -67774.29    -39913.918   -79250.57    -71256.95    -15949.062
  -35654.242   -58776.08    -94103.72    -57683.992   -51839.375
  -18169.535    -6839.1035  -68773.07    -41112.28    -22509.2
  -35473.77    -66644.05  ], shape=(32,), dtype=float32)
tf.Tensor(
[ -35476.676 -121289.87  -102680.734  -84481.24   -93406.29  -171691.95
  -8504

tf.Tensor(
[ -67526.    -113595.52   -96267.11  -159668.19   -88177.1   -161048.83
  -79806.97  -136768.98   -93085.54  -122740.18  -113917.12  -147597.31
 -110362.32   -39634.766 -179767.77  -101818.08   -56761.082 -116196.03
 -104040.05   -68511.375  -51519.797  -86630.15  -139495.61   -82582.47
  -73636.59   -76309.45   -73319.18   -98252.07   -60438.613 -199782.86
  -51334.99   -98165.06 ], shape=(32,), dtype=float32)
tf.Tensor(
[ -67526.    -113595.52   -96267.11  -159668.19   -88177.1   -161048.83
  -79806.97  -136768.98   -93085.54  -122740.18  -113917.12  -147597.31
 -110362.32   -39634.766 -179767.77  -101818.08   -56761.082 -116196.03
 -104040.05   -68511.375  -51519.797  -86630.15  -139495.61   -82582.47
  -73636.59   -76309.45   -73319.18   -98252.07   -60438.613 -199782.86
  -51334.99   -98165.06 ], shape=(32,), dtype=float32)
tf.Tensor(
[ -42339.793  -72881.18   -66261.41  -100813.44   -58095.418 -106651.45
  -51989.406  -86235.49   -61140.176  -78024.42   -72656.74   -91

tf.Tensor(
[ -41267.266  -71282.33   -64922.973  -98166.81   -56920.824 -104141.45
  -50948.906  -84082.39   -59365.402  -76327.11   -70803.44   -89123.84
  -68931.17   -26191.125 -110191.76   -64644.793  -38231.59   -75530.414
  -67998.05   -80534.27   -33827.3    -55581.91   -89577.86   -55277.17
  -49691.414  -82662.33   -45258.12   -66164.805  -39208.668 -120885.78
  -33611.543  -63184.707], shape=(32,), dtype=float32)
tf.Tensor(
[ -41267.266  -71282.33   -64922.973  -98166.81   -56920.824 -104141.45
  -50948.906  -84082.39   -59365.402  -76327.11   -70803.44   -89123.84
  -68931.17   -26191.125 -110191.76   -64644.793  -38231.59   -75530.414
  -67998.05   -80534.27   -33827.3    -55581.91   -89577.86   -55277.17
  -49691.414  -82662.33   -45258.12   -66164.805  -39208.668 -120885.78
  -33611.543  -63184.707], shape=(32,), dtype=float32)
tf.Tensor(
[ -62609.824 -105133.984  -90409.52  -147636.17   -82221.016 -149995.14
  -74626.85  -127304.56   -86339.5   -114087.14  -105545.06  -1

...

tf.Tensor(
[-108305.68   -78670.79   -50342.76   -89814.59   -68275.53  -100837.336
 -122016.04   -57212.875  -57362.4    -76470.11  -100852.63   -70846.695
  -97070.03  -147892.78  -121706.914  -65518.2    -66390.85   -78628.164
 -101273.34   -98728.44  -100444.57   -44667.715  -85916.4    -67872.055
 -118325.98   -95751.836  -66305.1    -86062.73   -81488.74   -57144.07
 -103606.664 -129782.06 ], shape=(32,), dtype=float32)
tf.Tensor(
[ -84636.35   -62474.01   -40272.586  -72055.59   -55274.78   -78875.7
  -95315.19   -60796.832  -45524.96   -64646.594  -79361.11   -59990.95
  -75512.61  -115705.23   -90924.44   -51737.02   -51254.35   -60214.65
  -76914.75   -78592.2    -79326.76   -46504.43   -67717.06   -52206.59
  -88033.94   -69914.65   -51535.79   -69686.26   -65168.67   -48483.74
  -77402.84   -94871.71 ], shape=(32,), dtype=float32)
tf.Tensor(
[ -84636.35   -62474.01   -40272.586  -72055.59   -55274.78   -78875.7
  -95315.19   -60796.832  -45524.96   -64646.594  -79361.11   -

tf.Tensor(
[ -83482.06   -80144.266  -52183.133  -90995.57   -54702.434  -77753.99
 -127948.84   -73025.75   -44835.99   -63852.13   -78446.11   -59381.67
  -74336.08  -113669.99   -89518.586  -51007.496  -50400.594  -80513.555
  -76141.8    -77600.7   -101225.734  -46104.43   -89954.445  -51107.633
  -86595.35   -68508.734  -50864.9    -88750.1    -64355.137  -57333.188
  -76202.516  -92952.2  ], shape=(32,), dtype=float32)
tf.Tensor(
[-102890.6    -93682.1    -60400.5   -105524.7    -65333.332  -95620.74
 -149752.62   -82373.02   -54281.445  -73509.74   -95973.96   -68207.92
  -92199.516 -139969.11  -114581.21   -62149.164  -62811.965  -95443.234
  -95798.35   -94169.58  -118381.664  -50403.09  -104825.95   -64023.523
 -111741.22   -89585.16   -63074.867 -101978.1    -77209.05   -64340.215
  -97342.67  -120630.336], shape=(32,), dtype=float32)
tf.Tensor(
[-102890.6    -93682.1    -60400.5   -105524.7    -65333.332  -95620.74
 -149752.62   -82373.02   -54281.445  -73509.74   -95973.96

tf.Tensor(
[ -82766.95   -97640.41   -62512.562  -90434.87   -68457.695  -76954.414
 -127042.43   -72617.484  -56424.71   -63397.965  -98442.74   -70337.3
 -116980.734 -112895.17   -88298.125  -50682.305  -81030.766  -79713.78
  -75216.71   -76905.266 -100491.97   -57194.145  -89339.91   -65816.65
  -85254.41   -67557.8    -50222.42   -88288.46   -80924.51   -65609.92
  -75229.85   -92069.26 ], shape=(32,), dtype=float32)
tf.Tensor(
[ -82766.95   -97640.41   -62512.562  -90434.87   -68457.695  -76954.414
 -127042.43   -72617.484  -56424.71   -63397.965  -98442.74   -70337.3
 -116980.734 -112895.17   -88298.125  -50682.305  -81030.766  -79713.78
  -75216.71   -76905.266 -100491.97   -57194.145  -89339.91   -65816.65
  -85254.41   -67557.8    -50222.42   -88288.46   -80924.51   -65609.92
  -75229.85   -92069.26 ], shape=(32,), dtype=float32)
tf.Tensor(
[ -99776.1   -109455.82   -69822.34  -103139.66   -77735.32   -92467.516
 -146044.14   -80753.336  -64805.36   -71757.66  -113611.625  -7

tf.Tensor(
[ -82148.22  -112583.92   -72099.875 -106155.89   -79672.09   -76532.37
 -126645.48   -83176.43   -77332.78   -63262.547  -98161.52   -70300.125
 -116820.7   -112077.734  -87723.9    -62714.43   -80780.05   -96449.21
  -74565.88   -76439.16  -100150.03   -61809.227 -140004.84   -80549.27
  -84805.42   -67142.164  -49920.387 -102194.39   -94625.484  -80850.836
  -74724.35   -91345.71 ], shape=(32,), dtype=float32)
tf.Tensor(
[ -97684.61  -123410.445  -78834.74  -117869.41   -88330.805  -90824.734
 -144275.75   -90569.01   -84899.875  -71054.39  -112272.71   -77456.61
 -131331.84  -133305.88  -107732.55   -71908.55   -90851.53  -108732.2
  -90096.16   -89671.5   -113977.625  -65281.09  -151946.61   -90961.734
 -104889.734  -84087.2    -59602.31  -112975.33  -104934.266  -86620.9
  -91605.875 -113754.836], shape=(32,), dtype=float32)
tf.Tensor(
[ -97684.61  -123410.445  -78834.74  -117869.41   -88330.805  -90824.734
 -144275.75   -90569.01   -84899.875  -71054.39  -112272.71   

KeyboardInterrupt: 

In [ ]:
a = tf.constant([[1.08,2.0,-3.0],[-4,0,-1]])
b = tf.math.sign(tf.math.sign(a)-1)+1
b